In [173]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [174]:
df1 = pd.read_csv(r'D:\singapore\ResaleFlatPricesBasedonApprovalDate19901999.csv')
df2 = pd.read_csv(r'D:\singapore\ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv')
df3 = pd.read_csv(r'D:\singapore\ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv')
df4 = pd.read_csv(r'D:\singapore\ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv')
df5 = pd.read_csv(r'D:\singapore\ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')
df=pd.concat([df1,df2,df3,df4,df5], ignore_index=True)


In [175]:
def convert_to_years(row):
    if pd.notna(row) and isinstance(row, str):
        if 'years' in row:
            parts = row.split()
            years = float(parts[0])
            months = float(parts[2]) if len(parts) > 2 else 0
            return years + months / 12
    return row



In [176]:
df['remaining_lease'] =df['remaining_lease'].apply(convert_to_years)

In [177]:
# fill null values in remaining_lease with mode beacuse null value more than 25% it will remove affected in accuracy
df['remaining_lease']=df['remaining_lease'].fillna(df['remaining_lease'].mode()[0])


In [178]:
#Month column

# Separate the year and month of resale beacuse this is time series Data
df[['resale_year', 'resale_month']] =df['month'].str.split('-', expand=True)

# drop the resale month column
df.drop('month', axis = 1, inplace= True)


In [179]:
# Split the 'Range' column into lower and upper bounds
df[['storey_lower_bound', 'storey_upper_bound']] = df['storey_range'].str.split(' TO ', expand=True)

# Convert the lower and upper bounds to numeric values
df['storey_lower_bound'] = pd.to_numeric(df['storey_lower_bound'])
df['storey_upper_bound'] = pd.to_numeric(df['storey_upper_bound'])
df.drop('storey_range', axis= 1, inplace= True)

In [180]:
df.head(2)

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,resale_year,resale_month,storey_lower_bound,storey_upper_bound
0,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,31.0,IMPROVED,1977,9000.0,68.0,1990,01,10,12
1,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,31.0,IMPROVED,1977,6000.0,68.0,1990,01,4,6


In [181]:
#convert because we only split the year month to string only

df['block'] = pd.to_numeric(df['block'], errors='coerce')
df['resale_year'] = pd.to_numeric(df['resale_year'], errors='coerce')
df['resale_month'] = pd.to_numeric(df['resale_month'], errors='coerce')
df['block'] = df['block'].fillna(df['block'].mode()[0])

In [182]:
# Replace the specified column to avoid duplicate
df['flat_type'] = df['flat_type'].str.replace('MULTI-GENERATION', 'MULTI GENERATION')

In [183]:
df['flat_model'] = df['flat_model'].replace({'MODEL A':'Model A','NEW GENERATION':'New Generation',
                                             'IMPROVED':'Improved','PREMIUM APARTMENT':'Premium Apartment',
                                             'MULTI GENERATION':'Multi Generation ','2-ROOM':'2-room',
                                            'SIMPLIFIED':'Simplified','STANDARD':'Standard',
                                             'MAISONETTE':'Maisonette','MULTI GENERATION':'Multi Generation',
                                             'APARTMENT':'Apartment','MODEL A-MAISONETTE':'Model A-Maisonette',
                                             'TERRACE':'Terrace','IMPROVED-MAISONETTE':'Improved-Maisonette'})

In [184]:
# remove duplicates

df.drop_duplicates(inplace=True)


In [185]:
#Using IQR method to eliminate outliers

#block

Q1 = df['block'].quantile(0.25)
Q3 = df['block'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df.block = df.block.clip(lower_bound , upper_bound)

#floor_area_sqm

Q1 = df['floor_area_sqm'].quantile(0.25)
Q3 = df['floor_area_sqm'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df.floor_area_sqm = df.floor_area_sqm.clip(lower_bound , upper_bound)

#resale_price

Q1 = df['resale_price'].quantile(0.25)
Q3 = df['resale_price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df.resale_price = df.resale_price.clip(lower_bound , upper_bound)

#remaining_lease

Q1 = df['remaining_lease'].quantile(0.25)
Q3 = df['remaining_lease'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df.remaining_lease = df.remaining_lease.clip(lower_bound , upper_bound)

#lease_commence_date

Q1 = df['lease_commence_date'].quantile(0.25)
Q3 = df['lease_commence_date'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df.lease_commence_date = df.lease_commence_date.clip(lower_bound , upper_bound)

#storey_lower_bound

Q1 = df['storey_lower_bound'].quantile(0.25)
Q3 = df['storey_lower_bound'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df.storey_lower_bound = df.storey_lower_bound.clip(lower_bound , upper_bound)

#storey_upper_bound

Q1 = df['storey_upper_bound'].quantile(0.25)
Q3 = df['storey_upper_bound'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df.storey_upper_bound = df.storey_upper_bound.clip(lower_bound , upper_bound)



In [186]:

le= LabelEncoder()
streets = df['street_name'].unique()
# Dictionary Comprehension
streets_mapping = {street: idx + 1 for idx, street in enumerate(streets)}
# alter the column with the mapped numeric numbers for training
df['street_name'] = df['street_name'].map(streets_mapping)
df.head(2)

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,resale_year,resale_month,storey_lower_bound,storey_upper_bound
0,ANG MO KIO,1 ROOM,309.0,1,31.0,Improved,1977.0,9000.0,68.0,1990,1,10,12
1,ANG MO KIO,1 ROOM,309.0,1,31.0,Improved,1977.0,6000.0,68.0,1990,1,4,6


In [187]:
category_mapping = {
    '1 ROOM': 1,
    '2 ROOM': 2,
    '3 ROOM': 3,
    '4 ROOM': 4,
    '5 ROOM': 5,
    'EXECUTIVE': 6,
    'MULTI GENERATION': 7
}

# alter the column with the mapped numbers
df['flat_type'] =df['flat_type'].map(category_mapping)
df.head(2)

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,resale_year,resale_month,storey_lower_bound,storey_upper_bound
0,ANG MO KIO,1,309.0,1,31.0,Improved,1977.0,9000.0,68.0,1990,1,10,12
1,ANG MO KIO,1,309.0,1,31.0,Improved,1977.0,6000.0,68.0,1990,1,4,6


In [188]:
# Create a mapping of 'flat_model' values to numbers
flat_model_mapping = {
    'IMPROVED': 1,
    'NEW GENERATION': 2,
    'MODEL A': 3,
    'STANDARD': 4,
    'SIMPLIFIED': 5,
    'MODEL A-MAISONETTE': 6,
    'APARTMENT': 7,
    'MAISONETTE': 8,
    'TERRACE': 9,
    '2-ROOM': 10,
    'IMPROVED-MAISONETTE': 11,
    'MULTI GENERATION': 12,
    'PREMIUM APARTMENT': 13,
    'Improved': 14,
    'New Generation': 15,
    'Model A': 16,
    'Standard': 17,
    'Apartment': 18,
    'Simplified': 19,
    'Model A-Maisonette': 20,
    'Maisonette': 21,
    'Multi Generation': 22,
    'Adjoined flat': 23,
    'Premium Apartment': 24,
    'Terrace': 25,
    'Improved-Maisonette': 26,
    'Premium Maisonette': 27,
    '2-room': 28,
    'Model A2': 29,
    'DBSS': 30,
    'Type S1': 31,
    'Type S2': 32,
    'Premium Apartment Loft': 33,
    '3Gen': 34
}

# Map 'flat_model' values to numbers
df['flat_model'] =df['flat_model'].map(flat_model_mapping)


In [189]:
# Create a mapping of 'town' values to numbers
town_mapping = {
    'ANG MO KIO': 1,
    'BEDOK': 2,
    'BISHAN': 3,
    'BUKIT BATOK': 4,
    'BUKIT MERAH': 5,
    'BUKIT TIMAH': 6,
    'CENTRAL AREA': 7,
    'CHOA CHU KANG': 8,
    'CLEMENTI': 9,
    'GEYLANG': 10,
    'HOUGANG': 11,
    'JURONG EAST': 12,
    'JURONG WEST': 13,
    'KALLANG/WHAMPOA': 14,
    'MARINE PARADE': 15,
    'QUEENSTOWN': 16,
    'SENGKANG': 17,
    'SERANGOON': 18,
    'TAMPINES': 19,
    'TOA PAYOH': 20,
    'WOODLANDS': 21,
    'YISHUN': 22,
    'LIM CHU KANG': 23,
    'SEMBAWANG': 24,
    'BUKIT PANJANG': 25,
    'PASIR RIS': 26,
    'PUNGGOL': 27
}

# Map 'town' values to numbers
df['town'] = df['town'].map(town_mapping)

# Display the DataFrame
df.head(4)

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,resale_year,resale_month,storey_lower_bound,storey_upper_bound
0,1,1,309.0,1,31.0,14,1977.0,9000.0,68.0,1990,1,10,12
1,1,1,309.0,1,31.0,14,1977.0,6000.0,68.0,1990,1,4,6
2,1,1,309.0,1,31.0,14,1977.0,8000.0,68.0,1990,1,10,12
3,1,1,309.0,1,31.0,14,1977.0,6000.0,68.0,1990,1,7,9


In [193]:
df.isna().sum()

town                   0
flat_type              0
block                  0
street_name            0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
resale_year            0
resale_month           0
storey_lower_bound     0
storey_upper_bound     0
dtype: int64

In [196]:
corrn=df.corr()
corrn 

,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,resale_year,resale_month,storey_lower_bound,storey_upper_bound
town,1.000000,0.208313,0.050267,0.586991,0.201102,0.163781,0.333020,0.123391,NaN,0.123874,0.003880,-0.019096,-0.018833
flat_type,0.208313,1.000000,0.127806,0.386564,0.954856,0.242332,0.407707,0.599264,NaN,0.100757,0.003379,0.077410,0.077741
block,0.050267,0.127806,1.000000,0.046152,0.187808,-0.003256,0.002743,-0.083383,NaN,-0.116867,0.002348,-0.124889,-0.124925
street_name,0.586991,0.386564,0.046152,1.000000,0.355429,0.354270,0.737482,0.379249,NaN,0.317400,0.006034,0.081871,0.082506
floor_area_sqm,0.201102,0.954856,0.187808,0.355429,1.000000,0.200253,0.385671,0.573701,NaN,0.062463,0.002847,0.043664,0.043853
flat_model,0.163781,0.242332,-0.003256,0.354270,0.200253,1.000000,0.355596,0.245786,NaN,0.162440,0.001489,0.020114,0.020470
lease_commence_date,0.333020,0.407707,0.002743,0.737482,0.385671,0.355596,1.000000,0.544094,NaN,0.431438,0.003129,0.154171,0.154567
resale_price,0.123391,0.599264,-0.083383,0.379249,0.573701,0.245786,0.544094,1.000000,NaN,0.684518,0.004631,0.210021,0.212665
remaining_lease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
resale_year,0.123874,0.100757,-0.116867,0.317400,0.062463,0.162440,0.431438,0.684518,NaN,1.000000,-0.026427,0.096775,0.098803


In [198]:
#Split data to test and train
X = df.drop('resale_price', axis= 1)
y = df['resale_price']
#splitting the data into training and testing sets with the ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=72)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(741581, 12) (185396, 12) (741581,) (185396,)


In [199]:
#scalling the data using Standard scaler

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled ,X_test_scaled

(array([[-1.62405716, -1.06998617, -0.63507496, ...,  1.59788163,
         -0.58096578, -0.58434491],
        [-1.62405716, -0.01555525,  0.12606584, ...,  0.4251786 ,
          1.44064851,  1.43768023],
        [-0.34171246, -0.01555525,  0.98234923, ...,  0.71835436,
          0.76677708,  0.76367185],
        ...,
        [ 1.45357011,  1.03887568, -0.18980759, ..., -0.16117292,
          1.44064851,  1.43768023],
        [ 0.29945989, -0.01555525, -0.76066319, ...,  0.4251786 ,
          1.44064851,  1.43768023],
        [-1.23935375, -0.01555525,  0.44574497, ...,  0.71835436,
         -0.58096578, -0.58434491]]),
 array([[-1.11111928, -1.06998617, -1.10317654, ...,  1.01153012,
         -0.58096578, -0.58434491],
        [ 1.32533564,  1.03887568,  0.14128866, ..., -1.62705171,
          0.76677708,  0.76367185],
        [-0.08524352,  1.03887568, -1.10698225, ..., -0.16117292,
          1.44064851,  1.43768023],
        ...,
        [-1.49582269, -1.06998617, -0.94333698, ...,  

In [206]:
#Model_Fit
#Method-1 Linear Regression

linear_model = LinearRegression(fit_intercept=True) 
linear_model.fit(X_train,y_train) 
print(linear_model.predict(X_test))     
print(y_test)
y_pred = linear_model.predict(X_test)

[184766.38886489 385222.73538425 530671.48321082 ... 242607.75496126
 238456.67437534 285243.38612822]
313374    163000.0
456117    292000.0
700084    488000.0
725160    440000.0
883698    566000.0
            ...   
720320    595000.0
34855     226000.0
447115    160000.0
16535     138000.0
474643    166800.0
Name: resale_price, Length: 185396, dtype: float64


In [204]:
#Model Score
linear_model.score(X_test, y_test)



0.7935376938418925

In [205]:
#cross validating  linear_model
from sklearn.model_selection import cross_val_score
cross_val_linear_model=cross_val_score(linear_model,X_train,y_train,cv=10).mean()
cross_val_linear_model

np.float64(0.7923128542494796)

In [212]:
mae_ln = mean_absolute_error(y_test, y_pred)
mse_ln = mean_squared_error(y_test, y_pred)
rmse_ln = np.sqrt(mse_ln)
r2_ln = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE)    :", mae_ln)
print("Mean Squared Error (MSE)     :", mse_ln)
print("Root Mean Square Error (RMSE):", rmse_ln)
print("R-squared (R2) Score         :", r2_ln)

Mean Absolute Error (MAE)    : 57160.2001059049
Mean Squared Error (MSE)     : 5514210796.373493
Root Mean Square Error (RMSE): 74257.73223290281
R-squared (R2) Score         : 0.7935376938418925


In [213]:
# Create and train the K Nearest Neighbor Regression model
knn_regressor = KNeighborsRegressor(n_neighbors=5)  # You may need to adjust the number of neighbors
knn_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set using KNN model
y_pred_knn = knn_regressor.predict(X_test_scaled)

# Calculate Mean Absolute Error (MAE)
mae_knn = mean_absolute_error(y_test, y_pred_knn)

# Calculate Mean Squared Error (MSE)
mse_knn = mean_squared_error(y_test, y_pred_knn)

# Calculate Root Mean Squared Error (RMSE)
rmse_knn = np.sqrt(mse_knn)

# Calculate R-squared (R2) Score
r2_knn = r2_score(y_test, y_pred_knn)

print("Mean Absolute Error (MAE)    :", mae_knn)
print("Mean Squared Error (MSE)     :", mse_knn)
print("Root Mean Square Error (RMSE):", rmse_knn)
print("R-squared (R2) Score         :", r2_knn)

Mean Absolute Error (MAE)    : 28933.806090929684
Mean Squared Error (MSE)     : 1779051675.0732834
Root Mean Square Error (RMSE): 42178.80599392642
R-squared (R2) Score         : 0.9333889970525538


In [214]:
# Create and train the Decision Tree Regressor model
dt_regressor = DecisionTreeRegressor(max_depth=20, random_state=11)
dt_regressor.fit(X_train, y_train)

# Make predictions on the test set using Decision Tree model
y_pred_dt = dt_regressor.predict(X_test)

# Calculate Mean Absolute Error (MAE)
mae_dt = mean_absolute_error(y_test, y_pred_dt)

# Calculate Mean Squared Error (MSE)
mse_dt = mean_squared_error(y_test, y_pred_dt)

# Calculate Root Mean Squared Error (RMSE)
rmse_dt = np.sqrt(mse_dt)

# Calculate R-squared (R2) Score
r2_dt = r2_score(y_test, y_pred_dt)

print("Mean Absolute Error (MAE)    :", mae_dt)
print("Mean Squared Error (MSE)     :", mse_dt)
print("Root Mean Square Error (RMSE):", rmse_dt)
print("R-squared (R2) Score         :", r2_dt)

Mean Absolute Error (MAE)    : 17585.002847585256
Mean Squared Error (MSE)     : 684777068.1241968
Root Mean Square Error (RMSE): 26168.245415468664
R-squared (R2) Score         : 0.9743606731933262


In [215]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Create and train the Random Forest Regressor model
rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_regressor.predict(X_test)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate R-squared (R2) Score
rf_r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE)    :", mae)
print("Mean Squared Error (MSE)     :", mse)
print("Root Mean Square Error (RMSE):", rmse)
print("R-squared (R2) Score         :", rf_r2)

Mean Absolute Error (MAE)    : 13906.49661171711
Mean Squared Error (MSE)     : 404694803.5029418
Root Mean Square Error (RMSE): 20117.027700506398
R-squared (R2) Score         : 0.984847474007274
